# Outline

* Overview

* Pre-training the language model on Wikitext103 Dataset

  1. Preprocess Wikitext 103
  2. Make the encoder decoder architecture
  3. Add the custom layers for AWD LSTM.
  4. Add an optimizer
  5. Make the training loop
  6. Calculate Validation Accuracy
* Fine-tuning it on new data
   1. Preprocess the new data
   2. train the previously saved model again with varying learning rates
* Use the language model for classification
   1. take the encoder for the language model, add a classifier head on top of it and use it for classification

In [1]:
!pip install -q tensorflow-gpu==2.0.0-beta1
!pip install tensorflow-text

     |████████████████████████████████| 348.9MB 83kB/s 
     |████████████████████████████████| 501kB 45.9MB/s 
     |████████████████████████████████| 3.1MB 34.5MB/s 
     |████████████████████████████████| 6.2MB 1.4MB/s 
     |████████████████████████████████| 87.9MB 401kB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
import tensorflow as tf

import re
import html 

# Get Data for Language Model

In [3]:
# Get Wikitext 103
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip

--2019-08-11 21:07:04--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.95.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.95.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: ‘wikitext-103-v1.zip’

wikitext-103-v1.zip 100%[===================>] 181.42M  64.7MB/s    in 2.8s    

2019-08-11 21:07:07 (64.7 MB/s) - ‘wikitext-103-v1.zip’ saved [190229076/190229076]



In [4]:
!unzip wikitext-103-v1.zip

Archive:  wikitext-103-v1.zip
   creating: wikitext-103/
  inflating: wikitext-103/wiki.test.tokens  
  inflating: wikitext-103/wiki.valid.tokens  
  inflating: wikitext-103/wiki.train.tokens  


# Preprocess Data

Two steps : 

1. Apply a list of rules to text 
2. Then tokenize the text.

Note : Fastai applies these following rules before tokenization

defaults.text_pre_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces]

and these rules after tokenization. 

defaults.text_post_rules = [replace_all_caps, deal_caps]

We have implementations for all the pre_rules in preprocessing function that is applied before tokenization. 


In [0]:
BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'

In [0]:
def preprocess(x):
  x = x.strip().lower()
  
  def replace_rep(t):
    "Replace repetitions at the character level in text with the specified token"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
  
  # replace all the characters that occur more than 3 times with the xxrep token
  x = replace_rep(x)
  
  def replace_wrep(t):
    "Replace word repetitions in text with the specified token."
    def _replace_wrep(m):
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)
  
  # replaces all the words that occur more than 3 times in text gets replaced by xxwrep token
  x = replace_wrep(x)
  # fix html 
  re1 = re.compile(r'  +')
  x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace(' @,@ ',',').replace('\\', ' \\ ')
  x=re1.sub(' ', html.unescape(x))
  
  "Add spaces around / and # in `t`. \n" 
  x=re.sub(r'([/#\n])', r' \1 ', x)
  
  "Remove multiple spaces in `t`."
  
  x=re.sub(' {2,}', ' ', x)
  
  
  return x

# Create Tensorflow Dataset 

In [0]:
train_path = "wikitext-103/wiki.train.tokens"
valid_path = "wikitext-103/wiki.valid.tokens"
test_path = "wikitext-103/wiki.test.tokens"

In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text,target_text

In [0]:
def tokenize_data(path,num_words=None):
  data = open(path,'r').read()[0:100000]
  tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='unk',num_words=num_words)
  tokenizer.fit_on_texts([data])
  data = tokenizer.texts_to_sequences([data])[0]
  return tokenizer,data

In [0]:
#word_to_index = tokenizer.word_index
#index_to_word = {v:k for k,v in tokenizer.word_index.items()}

#def convert_to_text(line):
#  return ' '.join([index_to_word[i] for i in line])

In [0]:
def make_lm_dataset(data,seq_length,batch_size,buffer_size):
  
  dataset = tf.data.Dataset.from_tensor_slices(data)
  batch_set = dataset.batch(seq_length+1,drop_remainder = True)
  batch_set = batch_set.map(lambda x:split_input_target(x))
  batch_set = batch_set.shuffle(buffer_size)
  train_dataset = batch_set.skip(50).batch(batch_size,drop_remainder=True)  
  valid_dataset = batch_set.take(50).batch(batch_size,drop_remainder=True)
  return train_dataset,valid_dataset

In [0]:
tokenizer, tokenized_data = tokenize_data(train_path)
train_dataset, valid_dataset= make_lm_dataset(tokenized_data,seq_length=70,batch_size=10,buffer_size=10000)  # each batch will have 64 lines, each line has 70 words

In [0]:
vocab_size = len(tokenizer.word_index)+1

# Building the language model

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM
from tensorflow.keras import Model

In [0]:
class EncoderModel(Model):
  def __init__(self,vocab_size,emb_dim,seq_length,hid_dim):
    super(EncoderModel,self).__init__()
    self.embed = Embedding(vocab_size, emb_dim, input_length=seq_length)
    self.lstm1 = LSTM(hid_dim,return_sequences=True)
    self.lstm2 = LSTM(hid_dim,return_sequences=True)
    self.lstm3 = LSTM(hid_dim,return_sequences=True)
 
    
  def call(self,x):
    x = self.embed(x)
    x = self.lstm1(x)
    x = self.lstm2(x)
    x = self.lstm3(x)
    return x

In [0]:
class DecoderModel(Model):
  def __init__(self,vocab_size):
    super(DecoderModel,self).__init__()
    self.d2 = Dense(vocab_size,activation="softmax")
    
  def call(self,x):
    x = self.d2(x)
    return x

## Model

In [0]:
class MyModel(Model):
  def __init__(self,vocab_size,emb_dim,seq_length,hid_dim):
    super(MyModel, self).__init__()
    self.encoder = EncoderModel(vocab_size,emb_dim,seq_length,hid_dim)
    self.decoder = DecoderModel(vocab_size)

  def call(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x


In [0]:
model = MyModel(vocab_size=vocab_size,emb_dim=64,seq_length=70,hid_dim=128)

## Training Loop

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [18]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for source_sent, target_sent in train_dataset:
    train_step(source_sent, target_sent)
    
  for source_sent, target_sent in valid_dataset:
    test_step(source_sent,target_sent)
    
    
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

W0811 21:18:49.992311 139838467336064 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1, Loss: 7.949592113494873, Accuracy: 7.915966033935547, Test Loss: 7.315667629241943, Test Accuracy: 9.028571128845215
Epoch 2, Loss: 6.855672836303711, Accuracy: 8.369747161865234, Test Loss: 6.98321008682251, Test Accuracy: 9.028571128845215
Epoch 3, Loss: 6.654736042022705, Accuracy: 8.369747161865234, Test Loss: 7.094948768615723, Test Accuracy: 9.028571128845215
Epoch 4, Loss: 6.624736785888672, Accuracy: 8.369747161865234, Test Loss: 7.151017665863037, Test Accuracy: 9.028571128845215
Epoch 5, Loss: 6.611464500427246, Accuracy: 8.369747161865234, Test Loss: 7.183828830718994, Test Accuracy: 9.028571128845215


# Using Language Model as Classifier

In [20]:
imdb = tf.keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


In [0]:
my_enc = model.encoder

In [0]:
classifier = 